In [119]:
#library
import pandas as pd
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import nltk
import os
import numpy as np
from sklearn.metrics import confusion_matrix 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import gensim
from gensim import utils
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import multiprocessing as mp

from nltk.tokenize import sent_tokenize, word_tokenize
from random import shuffle

In [187]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#read file
data = pd.read_pickle('.../foody review.pkl')

In [4]:
#xem qua dữ liệu
data.head()

,user_link,user_name,brand_name,brand_link,review_content,score
0,https://www.foody.vn/thanh-vien/taiyo,Sunny,El Sol - Meat & Wine - Võ Thị Sáu,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,Quán steak hiếm hoi mà mình cực kì ưng ý từ lâ...,8.6
1,https://www.foody.vn/thanh-vien/viola_0309_tv,Nguyên Khánh,Busan Korean Food - Món Hàn Quốc - Đinh Tiên H...,https://www.foody.vn/ho-chi-minh/busan-korean-...,Vị trí dễ tìm. Giữ xe rất nhiệt tình. Dắt xe v...,7.4
2,https://www.foody.vn/thanh-vien/tuhao2509,Tú Hảo,TocoToco Bubble Tea - Cộng Hòa,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,"Địa điểm quán dễ tìm. Không gian cũng rộng, có...",6.4
3,https://www.foody.vn/thanh-vien/zip_zilip,Vy Nhật,Le Castella Viet Nam - Bánh Bông Lan Đài Loan ...,https://www.foody.vn/ho-chi-minh/le-castella-v...,Thấy bánh này đang hot rần rần mình cũng gọi t...,7.2
4,https://www.foody.vn/thanh-vien/tien97,Tien97,Tabletop - Boardgame & Coffee,https://www.foody.vn/ho-chi-minh/tabletop-boar...,Mỗi lần nghĩ tới boardgame thì sẽ nghĩ tới vô ...,10.0


In [5]:
data.shape

(174379, 6)

# Làm sạch dữ liệu

In [58]:
#remove punctutaiton fuction
import string
punc = string.punctuation
def remove_punc(s):
    return(s.translate(str.maketrans('','',string.punctuation)).lower())

In [188]:
reviews = data[['review_content', 'score']]
#Để cho đơn giản, giả định điểm số không nhỏ hơn 5 là tốt còn ngược lại là tệ
reviews['score'] = reviews['score'].apply(lambda x: 'good' if x >= 5 else 'bad')
reviews['review_content'] = reviews['review_content'].map(lambda x: remove_punc(x).lower())

In [189]:
#reset index
reviews = reviews.reset_index(drop=True)
reviews.head(10)

,review_content,score
0,quán steak hiếm hoi mà mình cực kì ưng ý từ lâ...,good
1,vị trí dễ tìm giữ xe rất nhiệt tình dắt xe vào...,good
2,địa điểm quán dễ tìm không gian cũng rộng có l...,good
3,thấy bánh này đang hot rần rần mình cũng gọi t...,good
4,mỗi lần nghĩ tới boardgame thì sẽ nghĩ tới vô ...,good
5,hôm bữa mình và bạn vô đây ăn gọi 2 phần nướng...,good
6,69k của tụi mình 2 bánh lớn 2 ly trà sữa 1 lớn...,good
7,trà sữa koi phải nói là quá ngon xuất sắc lun ...,good
8,bạn muốn giết thời gian bạn chán những lúc hẹ...,good
9,quán nhiều game vô cùng nhân viên nhiệt tình n...,good


# Sử dụng TF-IDF

In [190]:
#train and test split
x_train, x_test, y_train, y_test = train_test_split(reviews['review_content'], reviews['score'], test_size=0.2, random_state=42)

In [191]:
#Vectorize sử dụng TF-IDF
vectorizer = TfidfVectorizer(use_idf=True)
train_tfidf = vectorizer.fit_transform(x_train)
test_tfidf = vectorizer.transform(x_test)

In [192]:
#train model using logistic regression
logistic = LogisticRegression()
logistic.fit(train_tfidf , y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [193]:
# Đánh giá kết quả mô hình

"""
		Negative	Positive
Negative 	x		  x
Positive 	x		  x
"""

result = logistic.predict(test_tfidf)
print(confusion_matrix(y_test,result))

[[  804  1168]
 [  273 32631]]


## Xem qua một số trường hợp dự đoán nhầm

In [194]:
# Nhận nhầm thành nhận xét tốt
for i in range(200):
    if result[i] == 'good':
        if result[i] != y_test.values[i]:
            print(x_test.values[i], '\n')
            print('Actual:', y_test.values[i])
            print('Predicted: ',result[i], '\n')

lừa đảo

vào cả chục lần vẫn bảo đang bận không thể đặt được bánh làm ăn vậy sao mà lấu dài được nếu ko co bánh để đáp ứng được cho chương trình thì đừng chạy chương trình khuyến mãi treo đầu dê bán thịt chó nản ko thèm mua nữa 

Actual: bad
Predicted:  good 

kimbap fc

quán nằm trong hẻm cũng dễ tìm nhưng mình thấy k ngon mình kêu 1 phần gà phomai gà nhạt ăn như gà kho mì đen  rất nhạt ăn như không vị tô mì chỉ có mì và dưa leo sợi k nên gọi món này 1 soup bánh cá món này ngon thứ 2 trong các món mình gọi ăn the the chứ k cay đậm đà có thêm 2 con nghêu nữa  1 kimpap thịt chiên cuốn chặt tay món này ngon nhất nè kim chi thì chua nhưng ăn cũng ổn 

Actual: bad
Predicted:  good 

bún mắm  giả điếc

hôm nay mình mới đi ăn về và mình tức anh ách ấy mọi người ạ mình dẫn bạn bè tới đây ăn tổng cộng 5 người 1 anh trong nhóm gọi trước 1 tô thập cẩm mình thì tô bình thường còn mọi người vẫn chưa kịp gọi mà bà con của bà thuyên này đã lanh chan

Có thể thấy vấn đề phần nhiều là do thái độ phục vụ

In [195]:
# Nhận nhầm thành nhận xét kém
for i in range(600):
    if result[i] == 'bad':
        if result[i] != y_test.values[i]:
            print(x_test.values[i], '\n')
            print('Actual:', y_test.values[i])
            print('Predicted: ',result[i], '\n')

cuốn n roll restaurant  bà triệu

đồ ăn khá được đa dạng nhiều món cuốn điểm trừ là nhân viên quá kém không được đào tạo tử tế
1 mình gọi đồ ăn nhưng nhân viên quên ko order bếp chờ lâu quá mình yêu cầu nhân viên mang nước chấm cho mình trước em nhân viên nghe xong ngoảy mông đi và buông câu đã có đồ ăn đâu mà đòi nước chấm wtf 😡😡😡 cáu tập 1
2 rau rửa quá bẩn mình gọi em nhân viên phục vụ ra vẫn là em đó mình chỉ cái rau bẩn cho em em không nhìn ra và hỏi lại với vẻ khó chịu sao ah chị sau đó mình bảo rau nhà em rửa bẩn quá không một lời xin lỗi em bê luôn đĩa rau đi sau đó mang cho mình 1 khay rau khác vẫn không một lời xin lỗi cáu tập 2
cáu tập 3 trông xe viết phấn lên yên xe khách nhưng không có giẻ lau đi mình phải lôi giấy trong cốp hì hụi lau đi trông xe thản nhiên đứng nhìn
xin lỗi nhưng ăn có được đến mấy thì lần sau cũng không quay lại
 

Actual: good
Predicted:  bad 

cafe

không gian quán rất ok và thoáng mát nhưng mà pha chế nước thì dở tệ giá cả thì ko rẻ 

Actual: good
Pr

Nhãn được dự đoán có vẻ chính xác hơn so với nhãn thực tế. Điều này một phần do điểm số ban đầu là trung bình cộng của 5 yếu tố vị trí, giá cả, chất lượng, phục vụ, không gian. Vì vậy nếu khách hàng chỉ không hài lòng ở một yếu tố nhưng những yếu tố còn lại vẫn tốt thì điểm số trung bình vẫn sẽ cao.

# Sử dụng doc2vec

## Gán nhãn cho từng câu

In [146]:
review_label = list()
for index, row in reviews.iterrows():
    review_label.append(gensim.models.doc2vec.TaggedDocument(word_tokenize(reviews['review_content'][index]),
                                                             ['review_' + str(index)]))

In [147]:
review_label[0:4]

[TaggedDocument(words=['quán', 'steak', 'hiếm', 'hoi', 'mà', 'mình', 'cực', 'kì', 'ưng', 'ý', 'từ', 'lâu', 'nay', 'khi', 'quán', 'bên', 'nguyễn', 'siêu', 'bữa', 'nay', 'đi', 'thử', 'chi', 'nhánh', 'bên', 'võ', 'thị', 'sáu', 'mà', 'ưng', 'quá', 'quán', 'nằm', 'trong', 'hẻm', 'xe', 'hơi', 'hướng', 'võ', 'thị', 'sáu', '1', 'chiều', 'mình', 'đi', 'hai', 'bà', 'trưng', 'quẹo', 'xuống', 'hẻm', 'nằm', 'gần', 'quán', 'cafe', 'higland', 'nên', 'dễ', 'tìm', 'lắm', 'quán', 'có', 'bảng', 'hiệu', 'khá', 'nhỏ', 'nhưng', 'do', 'hẻm', 'cụt', 'nên', 'cũng', 'dễ', 'nhận', 'biết', 'quán', 'phục', 'vụ', 'niềm', 'nở', 'chào', 'đón', 'và', 'phục', 'vụ', 'chu', 'đáo', 'nói', 'tới', 'phần', 'thức', 'ăn', 'mình', 'gọi', 'mac', 'cheese', 'béo', 'ngập', 'phô', 'mai', '2', 'phần', 'thịt', 'steak', 'tendeloin', 'achentina', 'và', 'striploin', '200g', 'kèm', 'salad', 'phô', 'mai', 'phải', 'nói', 'là', 'rất', 'hài', 'lòng', 'vì', 'thịt', 'mềm', 'và', 'juicy', 'lắm', 'thêm', 'cái', 'khoản', 'tendeloin', 'nên', 'bò', 

In [148]:
## Tạo từ điển và train model
#build vocab and model
cores = mp.cpu_count()
model = Doc2Vec(min_count=1, window=10, size=100, workers=cores)
model.build_vocab(review_label)

In [149]:
for epoch in range(5):
    shuffle(review_label)
    model.train(review_label, total_examples=model.corpus_count, epochs=epoch)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

## Tạo train và test vector

In [150]:
ranges = len(data)
review_vec = np.zeros((ranges, 100))
for index in range(ranges):
    review_vec[index] = model.docvecs['review_' + str(index)]

In [151]:
review_vec

array([[-0.45649961, -0.02403747,  0.30128953, ...,  0.49583066,
         0.0561779 , -0.02354425],
       [-0.12095491, -0.81089294, -0.59928614, ..., -0.39983344,
         0.38917777, -0.59527576],
       [-0.12261435, -0.23277071, -0.15242317, ..., -0.45269626,
         0.20752737, -0.12751415],
       ..., 
       [ 0.22505678, -0.22037929, -0.40714124, ..., -0.39952335,
        -0.16156606,  0.09626034],
       [-0.2047025 , -0.30238736, -0.15120442, ..., -0.1457895 ,
        -0.25319514, -0.35750097],
       [-0.12789682, -0.15730767, -0.25094521, ..., -0.19948401,
         0.10544757, -0.08271421]])

In [152]:
x_train, x_test, y_train, y_test = train_test_split(review_vec, reviews['score'], test_size=0.2)

In [153]:
#train model using logistic regression
logistic = LogisticRegression()
logistic.fit(x_train , y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [155]:
# Đánh giá kết quả mô hình

"""
		Negative	Positive
Negative 	x		  x
Positive 	x		  x
"""

result = logistic.predict(x_test)
print(confusion_matrix(y_test,result))

[[  283  1635]
 [  216 32742]]


Kết quả này kém hơn so với sử dụng TF-IDF.